# Forecasting Consensus Expectations: Consumer Price Index

## Point + Directional + Distributional Forecasts

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
import seaborn as sns
import matplotlib.dates as mdates
import statsmodels.api as sm
import plotly.express as px

from tqdm.auto import tqdm
from scipy import stats, special
from scipy.optimize import brentq
from collections import defaultdict
from itertools import product
from scipy.stats import t as student_t, norm, binomtest, jarque_bera
from statsmodels.stats.diagnostic import het_breuschpagan

In [2]:
OUT_DIR = "../out"        
MOM_DF_FILE       = "cpi_mom_df.parquet"
MOM_DF_FULL_FILE  = "cpi_mom_df_full.parquet"
YOY_DF_FILE       = "cpi_yoy_df.parquet"
YOY_DF_FULL_FILE  = "cpi_yoy_df_full.parquet"

mom_df       = pd.read_parquet(os.path.join(OUT_DIR, MOM_DF_FILE),      engine="pyarrow")
mom_df_full  = pd.read_parquet(os.path.join(OUT_DIR, MOM_DF_FULL_FILE), engine="pyarrow")

yoy_df       = pd.read_parquet(os.path.join(OUT_DIR, YOY_DF_FILE),      engine="pyarrow")
yoy_df_full  = pd.read_parquet(os.path.join(OUT_DIR, YOY_DF_FULL_FILE), engine="pyarrow")

print("mom_df shape     :", mom_df.shape)
print("mom_df_full shape:", mom_df_full.shape)

print("yoy_df shape     :", yoy_df.shape)
print("yoy_df_full shape:", yoy_df_full.shape)

mom_df shape     : (57710, 11)
mom_df_full shape: (68150, 11)
yoy_df shape     : (39400, 11)
yoy_df_full shape: (46492, 11)


In [14]:
mom_df.head()

,release_date,period,median_survey,actual,economist,firm,forecast,asof,error,surprise,series
0,2006-01-18,2005-12-31,0.2000,0.2000,Adam Chester,Lloyds Bank PLC,NaN,NaT,NaN,0.0000,Core CPI M/M
1,2006-01-18,2005-12-31,0.2000,0.2000,Alessandro Truppia,Aletti Gestielle Sgr Spa,NaN,NaT,NaN,0.0000,Core CPI M/M
2,2006-01-18,2005-12-31,0.2000,0.2000,Alison Lynn Reaser,Point Loma Nazarene University,0.2000,2006-01-16,0.0000,0.0000,Core CPI M/M
3,2006-01-18,2005-12-31,0.2000,0.2000,Allan Von Mehren,Danske Bank AS,NaN,NaT,NaN,0.0000,Core CPI M/M
4,2006-01-18,2005-12-31,0.2000,0.2000,Andreas Busch,Bantleon AG,0.2000,2006-01-16,0.0000,0.0000,Core CPI M/M


In [15]:
yoy_df.head()

,release_date,period,median_survey,actual,economist,firm,forecast,asof,error,surprise,series
0,2006-01-18,2005-12-31,2.2000,2.2000,Adam Chester,Lloyds Bank PLC,NaN,NaT,NaN,0.0000,Core CPI Y/Y
1,2006-01-18,2005-12-31,2.2000,2.2000,Allan Von Mehren,Danske Bank AS,NaN,NaT,NaN,0.0000,Core CPI Y/Y
2,2006-01-18,2005-12-31,2.2000,2.2000,Andreas Busch,Bantleon AG,NaN,NaT,NaN,0.0000,Core CPI Y/Y
3,2006-01-18,2005-12-31,2.2000,2.2000,Andrew Gretzinger,Manulife Asset Management Limited,NaN,NaT,NaN,0.0000,Core CPI Y/Y
4,2006-01-18,2005-12-31,2.2000,2.2000,Aneta Markowska,Moore Capital Management LP,NaN,NaT,NaN,0.0000,Core CPI Y/Y


In [3]:
import pandas as pd
import numpy as np

# ------------------------------------------------------------------
# Helper:  count decimal places in a numeric Series
# ------------------------------------------------------------------
def count_decimals(series: pd.Series, max_scan: int = 1_000_000):
    """
    Return a value-counts Series telling you how many observations have
    0, 1, 2, … decimal places. NaNs are ignored.

    Parameters
    ----------
    series : pd.Series
        The numeric column to inspect.
    max_scan : int, optional
        Safety cap on how many rows to scan (defaults to 1 million).

    Examples
    --------
    >>> count_decimals(df['forecast'])
    0    378
    1    112
    2     45
    dtype: int64
    """
    # work on a copy of numeric values only
    vals = series.dropna().astype(float).to_numpy()[:max_scan]

    def decimals_of(x: float) -> int:
        # Convert to string, strip trailing zeros and the dot itself
        s = format(x, ".10f").rstrip("0").rstrip(".")
        return len(s.partition(".")[2])

    dec_counts = pd.Series([decimals_of(v) for v in vals]).value_counts().sort_index()
    dec_counts.index.name = "# decimal places"
    dec_counts.name = "count"
    return dec_counts

# ------------------------------------------------------------------
# Run on each CPI dataframe / column you care about
# ------------------------------------------------------------------
PANELS = {
    "Core CPI M/M  – forecast"      : mom_df["forecast"],
    "Core CPI M/M  – median_survey" : mom_df["median_survey"],
    "Core CPI Y/Y  – forecast"      : yoy_df["forecast"],
    "Core CPI Y/Y  – median_survey" : yoy_df["median_survey"],
}

for label, col in PANELS.items():
    print(f"\n{label}")
    print(count_decimals(col))



Core CPI M/M  – forecast
# decimal places
0      343
1    12955
2      401
3       81
4       13
5        1
Name: count, dtype: int64

Core CPI M/M  – median_survey
# decimal places
1    56550
2     1160
Name: count, dtype: int64

Core CPI Y/Y  – forecast
# decimal places
0     668
1    6474
2      70
3      48
4      11
Name: count, dtype: int64

Core CPI Y/Y  – median_survey
# decimal places
0     3349
1    34869
2      591
3      394
Name: count, dtype: int64


In [4]:
import pandas as pd
import numpy as np

# ------------------------------------------------------------
# helper: how many decimals does a single number have?
# ------------------------------------------------------------
def _decimals(x: float) -> int:
    """
    Return the number of digits after the decimal point for *one* float.
    NaNs → -1  (so they never match our >=4 filter)
    """
    if pd.isna(x):
        return -1
    s = format(float(x), ".10f").rstrip("0").rstrip(".")
    return len(s.partition(".")[2])

# ------------------------------------------------------------
# main inspection function
# ------------------------------------------------------------
def show_high_precision(df: pd.DataFrame,
                        col: str = "forecast",
                        min_decimals: int = 4):
    """
    Display all rows where `col` has ≥ `min_decimals` decimal places.

    Parameters
    ----------
    df : pd.DataFrame
    col : str
        Name of the numeric column to inspect (default "forecast").
    min_decimals : int
        Threshold of decimal places to flag (default 4).

    Returns
    -------
    pd.DataFrame
        The filtered rows, ready for further inspection.
    """
    mask = df[col].apply(_decimals) >= min_decimals
    flagged = df.loc[mask].copy()

    print(f"→ {len(flagged)} rows in '{col}' with ≥ {min_decimals} decimals\n")
    display_cols = ["release_date", "economist", "firm", col, "asof"]
    display(flagged[display_cols].sort_values("release_date").head(20))  # show first 20
    return flagged

# ------------------------------------------------------------
# EXAMPLE USAGE
# ------------------------------------------------------------
# For Core CPI M/M:
high_prec_mom = show_high_precision(mom_df)

# For Core CPI Y/Y:
high_prec_yoy = show_high_precision(yoy_df)


→ 14 rows in 'forecast' with ≥ 4 decimals



,release_date,economist,firm,forecast,asof
39579,2017-05-12,Kevin Cummins,Natwest Markets,0.22111,2017-05-08
53770,2024-06-12,John D Herrmann,Herrmann Forecasting LLC,0.25910,2024-06-12
54060,2024-07-11,John D Herrmann,Herrmann Forecasting LLC,0.18920,2024-07-10
54350,2024-08-14,John D Herrmann,Herrmann Forecasting LLC,0.18880,2024-08-14
54640,2024-09-11,John D Herrmann,Herrmann Forecasting LLC,0.24740,2024-09-11
54930,2024-10-10,John D Herrmann,Herrmann Forecasting LLC,0.23050,2024-10-09
55220,2024-11-13,John D Herrmann,Herrmann Forecasting LLC,0.22980,2024-11-13
55510,2024-12-11,John D Herrmann,Herrmann Forecasting LLC,0.26050,2024-12-10
55800,2025-01-15,John D Herrmann,Herrmann Forecasting LLC,0.27120,2025-01-14
56090,2025-02-12,John D Herrmann,Herrmann Forecasting LLC,0.30370,2025-02-11


→ 11 rows in 'forecast' with ≥ 4 decimals



,release_date,economist,firm,forecast,asof
36530,2024-06-12,John D Herrmann,Herrmann Forecasting LLC,3.5102,2024-06-12
36727,2024-07-11,John D Herrmann,Herrmann Forecasting LLC,3.4054,2024-07-10
36924,2024-08-14,John D Herrmann,Herrmann Forecasting LLC,3.2375,2024-08-14
37318,2024-10-10,John D Herrmann,Herrmann Forecasting LLC,3.1744,2024-10-09
37712,2024-12-11,John D Herrmann,Herrmann Forecasting LLC,3.2512,2024-12-10
37909,2025-01-15,John D Herrmann,Herrmann Forecasting LLC,3.2959,2025-01-14
38106,2025-02-12,John D Herrmann,Herrmann Forecasting LLC,3.1571,2025-02-11
38303,2025-03-12,John D Herrmann,Herrmann Forecasting LLC,3.2112,2025-03-11
38500,2025-04-10,John D Herrmann,Herrmann Forecasting LLC,3.1165,2025-04-09
38697,2025-05-13,John D Herrmann,Herrmann Forecasting LLC,2.9065,2025-05-12


## Static inverse-MSE

In [16]:
# ───────────────────────── settings ────────────────────────────
import numpy as np, pandas as pd
from itertools import product
from scipy import stats
from tqdm.auto import tqdm

contiguity_windows = [3, 6, 12, 24]                # rolling-window lengths
methods            = ["inverse_mse", "inverse_mae", "equal_weight"]
ridge              = 1e-6                          # stabiliser for inverse weights
start_date_eval    = "2010-01-01"                  # optional back-test start
# ───────────────────────────────────────────────────────────────

def run_static_panel(df_panel: pd.DataFrame, name: str):
    """
    Replicates the NFP static-weight grid search:
      • df_panel must have release_date / economist / forecast / actual / median_survey / error
      • returns (eval_rows_df, live_rows_df)
    """
    # trim evaluation period if desired
    df_panel = df_panel[df_panel["release_date"] >= start_date_eval].copy()

    dates = np.sort(df_panel["release_date"].unique())
    eval_rows, live_rows = [], []

    for window, method in tqdm(product(contiguity_windows, methods),
                               total=len(contiguity_windows)*len(methods),
                               desc=f"{name} grid"):

        preds = []                    # per-release records

        for idx in range(window, len(dates)):
            t = dates[idx]

            # ----- look-back slice -----
            hist = df_panel[df_panel["release_date"].isin(dates[idx-window:idx])]

            # economists with ALL forecasts inside window
            econs = hist.groupby("economist")["forecast"].apply(lambda s: s.notna().all())
            econs = econs[econs].index
            if econs.empty:
                continue

            # ----- compute weights -----
            if method == "equal_weight":
                w = pd.Series(1.0, index=econs)

            else:
                grp = hist[hist["economist"].isin(econs)].groupby("economist")["error"]
                score = grp.apply(lambda s:
                                  np.nanmean(s**2) if method == "inverse_mse"
                                  else np.nanmean(np.abs(s)))
                w = 1.0/(score + ridge)

            w /= w.sum()

            # ----- current forecasts -----
            cur = df_panel[(df_panel["release_date"] == t) &
                           (df_panel["economist"].isin(w.index))]
            f_t = cur.set_index("economist")["forecast"].dropna()
            w   = w.reindex(f_t.index).dropna()
            if w.empty:
                continue
            w /= w.sum()

            smart   = np.dot(w, f_t.loc[w.index])
            median  = df_panel.loc[df_panel["release_date"] == t,
                                   "median_survey"].iloc[0]
            actual  = df_panel.loc[df_panel["release_date"] == t,
                                   "actual"].iloc[0]

            preds.append((t, smart, median, actual))

        # nothing produced for this spec?
        if not preds:
            continue

        oos = pd.DataFrame(preds, columns=["date", "smart", "median", "actual"])

        # unreleased print (latest actual NaN) → stash live forecast
        unreleased = oos[oos["actual"].isna()]
        if not unreleased.empty:
            last = unreleased.iloc[-1]
            live_rows.append({"panel" : name,
                              "window": window,
                              "method": method,
                              "date"  : last["date"],
                              "smart" : last["smart"],
                              "median": last["median"],
                              "pred_dir": int(last["smart"] > last["median"]) })

        # evaluation on realised months only
        eval_df = oos.dropna(subset=["actual"]).copy()
        if eval_df.empty:
            continue

        eval_df["smart_err"]  = eval_df["smart"]  - eval_df["actual"]
        eval_df["median_err"] = eval_df["median"] - eval_df["actual"]
        eval_df["actual_dir"] = (eval_df["actual"] > eval_df["median"]).astype(int)
        eval_df["pred_dir"]   = (eval_df["smart"]  > eval_df["median"]).astype(int)

        obs         = len(eval_df)
        rmse_smart  = np.sqrt((eval_df["smart_err"]**2 ).mean())
        rmse_median = np.sqrt((eval_df["median_err"]**2).mean())

        d       = eval_df["smart_err"]**2 - eval_df["median_err"]**2
        dm_stat = d.mean()/d.std(ddof=1)*np.sqrt(obs)
        dm_p    = 2*(1-stats.norm.cdf(abs(dm_stat)))

        hits     = (eval_df["actual_dir"] == eval_df["pred_dir"]).astype(int)
        hit_rate = hits.mean()
        binom_p  = stats.binomtest(hits.sum(), obs, 0.5).pvalue
        p1, p2   = eval_df["pred_dir"].mean(), eval_df["actual_dir"].mean()
        c_joint  = (eval_df["pred_dir"] & eval_df["actual_dir"]).mean()
        pt_stat  = (c_joint - p1*p2) / np.sqrt(p1*p2*(1-p1)*(1-p2)/obs)
        pt_p     = 2*(1-stats.norm.cdf(abs(pt_stat)))

        eval_rows.append({"panel":name,"window":window,"method":method,
                          "obs":obs,"RMSE_smart":rmse_smart,"RMSE_median":rmse_median,
                          "HitRate":hit_rate,"Binom_p":binom_p,"PT_p":pt_p,"DM_p":dm_p})
    return pd.DataFrame(eval_rows), pd.DataFrame(live_rows)


# ────────────────────────── run panels ─────────────────────────
PANELS_COVID = {                     # “COVID-trimmed” set (match your mom_df / yoy_df)
    "Core CPI M/M": mom_df,
    "Core CPI Y/Y": yoy_df,
}
PANELS_FULL  = {                     # Full history
    "Core CPI M/M": mom_df_full,
    "Core CPI Y/Y": yoy_df_full,
}

res_covid, live_covid = [], []
for name, df in PANELS_COVID.items():
    e, l = run_static_panel(df.copy(), name)
    res_covid.append(e);  live_covid.append(l)
res_covid = pd.concat(res_covid, ignore_index=True)
live_covid = pd.concat(live_covid, ignore_index=True)

res_full, live_full = [], []
for name, df in PANELS_FULL.items():
    e, l = run_static_panel(df.copy(), name + " (Full)")
    res_full.append(e);   live_full.append(l)
res_full  = pd.concat(res_full,  ignore_index=True)
live_full = pd.concat(live_full, ignore_index=True)


# ───── robust-winner pick for each COVID panel (same logic) ────
def choose_winner(df_eval: pd.DataFrame, panel_name: str):
    sub = df_eval[df_eval["panel"] == panel_name]
    robust = sub[(sub["DM_p"]<0.10)&(sub["Binom_p"]<0.10)&(sub["PT_p"]<0.10)]
    winner = robust.loc[robust["RMSE_smart"].idxmin()] if not robust.empty \
             else sub.loc[sub["RMSE_smart"].idxmin()]
    return winner

winners = {p: choose_winner(res_covid, p) for p in PANELS_COVID.keys()}

# ──────────────────────────── PRINTS ───────────────────────────
pd.set_option("display.float_format", "{:.4f}".format)

print("\n=== COVID-filtered panel (historical) ===")
for p in PANELS_COVID.keys():
    print(f"\n--- {p} ---")
    print(res_covid[res_covid["panel"] == p]
          .sort_values(["window","method"])
          .to_string(index=False))

print("\n=== Full panel (historical) ===")
for p in PANELS_FULL.keys():
    print(f"\n--- {p} ---")
    print(res_full[res_full["panel"] == p+" (Full)"]
          .sort_values(["window","method"])
          .to_string(index=False))

# ---------- live forecasts (COVID set) ----------
if live_covid.empty:
    print("\nNo unreleased CPI prints – all actuals available.")
else:
    print("\n=== LIVE FORECASTS (COVID-filtered data) ===")
    for p, win_row in winners.items():
        w, m = int(win_row["window"]), win_row["method"]
        live_rows = live_covid[(live_covid["panel"] == p) &
                               (live_covid["method"] == m)].sort_values("window")
        if live_rows.empty:
            continue
        print(f"\n>>> {p}   •   method = {m}")
        for _, row in live_rows.iterrows():
            direction = "Higher-than-median" if row["pred_dir"] else "Lower-than-median"
            print(f"[{int(row['window']):2d}-mo]  Date: {pd.to_datetime(row['date']).date()}  |  "
                  f"Smart: {row['smart']:.4f}  |  Median: {row['median']:.4f}  |  Signal: {direction}")


Core CPI M/M grid:   0%|          | 0/12 [00:00<?, ?it/s]

Core CPI Y/Y grid:   0%|          | 0/12 [00:00<?, ?it/s]

Core CPI M/M (Full) grid:   0%|          | 0/12 [00:00<?, ?it/s]

Core CPI Y/Y (Full) grid:   0%|          | 0/12 [00:00<?, ?it/s]


=== COVID-filtered panel (historical) ===

--- Core CPI M/M ---
       panel  window       method  obs  RMSE_smart  RMSE_median  HitRate  Binom_p   PT_p   DM_p
Core CPI M/M       3 equal_weight  147      0.0798       0.0836   0.7415   0.0000 0.0001 0.0421
Core CPI M/M       3  inverse_mae  147      0.0830       0.0836   0.7279   0.0000 0.0000 0.8565
Core CPI M/M       3  inverse_mse  147      0.0828       0.0836   0.7211   0.0000 0.0001 0.8017
Core CPI M/M       6 equal_weight  144      0.0799       0.0836   0.7431   0.0000 0.0001 0.0549
Core CPI M/M       6  inverse_mae  144      0.0799       0.0836   0.7222   0.0000 0.0006 0.0628
Core CPI M/M       6  inverse_mse  144      0.0794       0.0836   0.7222   0.0000 0.0006 0.0324
Core CPI M/M      12 equal_weight  138      0.0796       0.0837   0.7174   0.0000 0.0004 0.0410
Core CPI M/M      12  inverse_mae  138      0.0793       0.0837   0.7391   0.0000 0.0000 0.0304
Core CPI M/M      12  inverse_mse  138      0.0793       0.0837   0.731

In [ ]:
# ================================================================
#  A ▸ Single-model grid search:  Win ∈ {3,6,12}  ×  Method ∈ {equal, inv_MSE, inv_MAE, EWMA-inv_MSE}
#      → back-test (releases ≥ 2010-01-01)  →  RMSE & directional hit-rate
# ================================================================
import numpy as np, pandas as pd
from tqdm.auto import tqdm
from itertools import product
from scipy import stats

# --------------------- tune here ---------------------
WINDOWS      = [3, 6, 12]
METHODS      = ["equal", "inverse_mse", "inverse_mae", "ewma_inv_mse"]
EWMA_LAMBDA  = 0.90
RIDGE        = 1e-6
START_DATE   = "2010-01-01"

PANELS = {
    "Core CPI M/M": mom_df_full.copy(),
    "Core CPI Y/Y": yoy_df_full.copy(),
}
# -----------------------------------------------------

def build_weights(hist, method):
    """Return pd.Series(weights) indexed by economist."""
    if method == "equal":
        w = pd.Series(1.0, index=hist["economist"].unique())
    else:
        # error vector
        # err_tbl = (hist.groupby("economist")["error"]
        #                  .apply(lambda s: np.nan_to_num(s.values, nan=0.0)))
        err_tbl = hist.groupby("economist")["error"].apply(lambda s: s.values)
        if method == "ewma_inv_mse":
            n = len(hist["release_date"].unique())
            ew = EWMA_LAMBDA ** np.arange(n-1, -1, -1);  ew /= ew.sum()
            mse = err_tbl.apply(lambda e: np.dot(ew[-len(e):], e**2))
        elif method == "inverse_mse":
            mse = err_tbl.apply(lambda e: np.mean(e**2))
        else:  # inverse_mae
            mse = err_tbl.apply(lambda e: np.mean(np.abs(e)))
        w = 1.0 / (mse + RIDGE)
    return w / w.sum()

results = []

for panel_name, df_full in PANELS.items():
    df_full = df_full[df_full["release_date"] >= START_DATE].copy()
    dates   = np.sort(df_full["release_date"].unique())

    for W, meth in tqdm(product(WINDOWS, METHODS),
                        total=len(WINDOWS)*len(METHODS),
                        desc=f"{panel_name}  specs"):

        records = []           # per-release rows

        for idx in range(W, len(dates)):
            t = dates[idx]
            # look-back window
            hist = df_full[df_full["release_date"]
                           .between(dates[idx-W], dates[idx-1])]
            # economists with complete coverage
            elig = hist.groupby("economist")["forecast"].apply(lambda s: s.notna().all())
            econs = elig[elig].index
            if econs.empty:  continue

            w = build_weights(hist[hist["economist"].isin(econs)], meth)

            # current forecasts
            cur = df_full[(df_full["release_date"] == t) &
                          (df_full["economist"].isin(w.index))]
            f_t = cur.set_index("economist")["forecast"].dropna()
            w   = w.reindex(f_t.index).dropna()
            if w.empty:  continue
            w /= w.sum()

            smart   = np.dot(w, f_t.loc[w.index])
            median  = df_full.loc[df_full["release_date"] == t,
                                  "median_survey"].iloc[0]
            actual  = df_full.loc[df_full["release_date"] == t,
                                  "actual"].iloc[0]

            if np.isnan(actual):        # skip unreleased print
                continue
            records.append((smart, median, actual))

        # ------------- aggregate metrics -------------
        if not records:  continue
        rec = pd.DataFrame(records, columns=["smart","median","actual"])
        rec["smart_err"]  = rec["smart"]  - rec["actual"]
        rec["median_err"] = rec["median"] - rec["actual"]
        rec["hit"]        = (np.sign(rec["smart"]-rec["median"]) ==
                             np.sign(rec["actual"]-rec["median"])).astype(int)

        results.append({
            "Panel"       : panel_name,
            "Window"      : W,
            "Method"      : meth,
            "Obs"         : len(rec),
            "RMSE_smart"  : np.sqrt(np.mean(rec["smart_err"]**2)),
            "RMSE_median" : np.sqrt(np.mean(rec["median_err"]**2)),
            "HitRate"     : rec["hit"].mean()
        })

# ---------- display ----------
tbl = (pd.DataFrame(results)
       .sort_values(["Panel","Window","Method"])
       .reset_index(drop=True))
pd.set_option("display.float_format", "{:.4f}".format)
print("\n=== Single-method back-test results (≥2010 releases) ===")
display(tbl)


Core CPI M/M  specs:   0%|          | 0/12 [00:00<?, ?it/s]

Core CPI Y/Y  specs:   0%|          | 0/12 [00:00<?, ?it/s]


=== Single-method back-test results (≥2010 releases) ===


,Panel,Window,Method,Obs,RMSE_smart,RMSE_median,HitRate
0,Core CPI M/M,3,equal,183,0.1154,0.1186,0.4699
1,Core CPI M/M,3,ewma_inv_mse,183,0.1150,0.1186,0.4645
2,Core CPI M/M,3,inverse_mae,183,0.1154,0.1186,0.4645
3,Core CPI M/M,3,inverse_mse,183,0.1150,0.1186,0.4590
4,Core CPI M/M,6,equal,180,0.1159,0.1191,0.4722
5,Core CPI M/M,6,ewma_inv_mse,180,0.1136,0.1191,0.4389
6,Core CPI M/M,6,inverse_mae,180,0.1147,0.1191,0.4500
7,Core CPI M/M,6,inverse_mse,180,0.1136,0.1191,0.4389
8,Core CPI M/M,12,equal,174,0.1158,0.1202,0.4540
9,Core CPI M/M,12,ewma_inv_mse,174,0.1140,0.1202,0.4540
